# <center>Noto 7.5 Mw 2024 Earthquake Project</center>
<center>by: Haidir Jibran</center>
<center>haidirjibran@gmail.com</center>

Noto 4.5 Mw 2024 merupakan gempa bumi besar yang sangat merusak diwilayah utara kepulauan jepang.......

<b>1. Import Module and Library</b>

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as img

from obspy import UTCDateTime
from obspy.clients.fdsn import Client

<b>2. Mengambil Event Gempa Noto 7.5 Mw 2024</b>

Menurut Wikipedia, gempa ini terjadi pada wilayah semenanjung Noto Jepang yang berada pada koordinat 37.488N,137.271E yang terjadi pada 1 januari 2024 pukul 16.10 waktu setempat (07.10 UTC) https://en.wikipedia.org/wiki/2024_Noto_earthquake

Mendefinisikan variabel yang telah diketahui

In [ ]:
latitude = 37.488
longitude = 137.217
eventtime = UTCDateTime('2024-01-01T16:10:00+09')

In [ ]:
client = Client('IRIS')
event = client.get_events(starttime=eventtime-10, endtime=eventtime+10, longitude=longitude, latitude=latitude, maxradius=0.5)
print(event)
event.plot(projection='ortho', title="Noto 4.5 Mw 2024")
plt.close()

print(event[0].origins)

base on A topographic map of Japan :https://en.wikipedia.org/wiki/Japan#/media/File:Japan_topo_en.jpg range of coordinate japan is 125-150E and 25-50N

here we want to plot the whole event in japan for 5 years lately (aug 2019 - aug 2024) for know the sesmicity in japan and the position of the event we want to analisys

In [ ]:
os.system('./FetchEvent -s 2019-08-01,00:00:00 -e 2024-08-31,00:00:00 --lon 125:150 --lat 25:50 --mag 4:10 -o ./data/whole_japan.txt')
os.system('gmt gmtset PS_MEDIA = archE')
os.system('gmt pscoast -R123/152/23/52 -JM15 -Ba5g5/a5g5:."Japan Seismicity":WESN -Ggray -K > ./postscript/whole_japan.ps')
os.system("awk -F'|' '{print $4,$3}' ./data/whole_japan.txt | gmt psxy -R -J -Sc0.1 -Gred4 -O -K >> ./postscript/whole_japan.ps")
os.system('echo 137.264 37.489 | gmt psxy -R -J -Sa1 -Gyellow -O >> ./postscript/whole_japan.ps')
os.system('gmt psconvert ./postscript/whole_japan.ps -A -Tg')
plot_whole_japan=img.imread('./postscript/whole_japan.png')
plt.axis('off')
plt.imshow(plot_whole_japan)
plt.show()
plt.close()

In [ ]:
#os.system('./FetchEvent -s 2019-08-01,00:00:00 -e 2024-08-31,00:00:00 --lon 130:150 --lat 35:40 --mag 4:10 -o ./data/crosec.txt')
os.system('gmt gmtset PS_MEDIA = archE')
os.system('gmt pscoast -R123/152/23/52 -JM15 -Ba5g5/a5g5:."Cross Section":WESN -Ggray -K > ./postscript/crosec.ps')
os.system("awk -F'|' '{print $4,$3}' ./data/whole_japan.txt | gmt psxy -R -J -Sc0.1 -Gred4 -O -K >> ./postscript/crosec.ps")
os.system("awk -F'|' '{print $4,$3}' ./data/crosec.txt | gmt psxy -R -J -Sc0.1 -Gblue -O -K >> ./postscript/crosec.ps")
os.system('echo 137.264 37.489 | gmt psxy -R -J -Sa1 -Gyellow -O >> ./postscript/crosec.ps')
os.system('gmt psconvert ./postscript/crosec.ps -A -Tg')
plot_crosec=img.imread('./postscript/crosec.png')
plt.axis('off')
plt.imshow(plot_crosec)
plt.show()
plt.close()

os.system("awk -F'|' '{print $4,$5*-1}' ./data/crosec.txt | gmt psxy -R130/150/-500/0 -JX10/4 -Ba5g5:Longitude:/a100g100:'Depth (Km)'::.'Cross Section':WeSn -Sc0.1 -Gblue -K > ./postscript/xsec.ps")
os.system('echo 137.264 -10 | gmt psxy -R -J -Sa0.5 -Gyellow -O >> ./postscript/xsec.ps')
os.system('gmt psconvert ./postscript/xsec.ps -A -Tg')
plot_xsec=img.imread('./postscript/xsec.png')
plt.axis('off')
plt.imshow(plot_xsec)
plt.show()